### Test data set

> Length = 1500bps   
> Flank  = 100bps   
> Minin  = 50bps   
> Minex  = 50bps   
> Maxs   = 3

### Idea1
using different nested loop so we could do more precise unsatisfied intron and exon remove

In [ ]:
def gtag_sites(seq, flank, minex):
    dons = []
    accs = []
    for i in range(flank + minex, len(seq) -flank -minex -1):
        if seq[i:i+2]   == 'GT': dons.append(i)
        if seq[i-1:i+1] == 'AG': accs.append(i)
    return dons, accs

using original `gtag extracting` doesn't work that well
original one gives out dons and accs in version of list

> dons = []   
> accs = []   

since for using up a for loop to make all possible combination, we also need the enumerate so that each time when the nested loop pass down to the next combination
there would require more memory and time complexity for computer to keep track of those numbers

In [ ]:
dons, accs = gtag_sites(seq, 100, 50)

for i1, d1 in enumerate(dons):
    if d1 - 100 -1 < 50: continue
    
    for j1, a1 in enumerate(accs):
        if a1 - d1 < 50: continue
        
        for i2, d2 in enumerate(dons[i1:]):
            if d2 - a1 < 50: continue
            
            for j2, a2 in enumerate(accs[j1:]):
                if a2 - d2 < 50: continue
                
                for i3, d3 in enumerate(dons[i2:]):
                    if d3 - a2 < 50: continue
            
                    for a3 in accs[j2:]:
                        if a3 - d3 < 50: continue
                        if a3 - 100 - 1 < 50:  continue
                        print(d1, a1, d2, a2, d3, a3)

Performance: 110.64s user 0.56s system 99% cpu 1:51.22 total

### Idea2   
using dictionary instead of list for donor and acceptor site

In [ ]:
def gtag_sites(seq, flank, minex):
    dons = {}
    accs = {}
    d = 0
    a = 0
    for i in range(flank + minex, len(seq) -flank -minex -1):
        if seq[i:i+2]   == 'GT': 
            d += 1
            dons[d] = i
        if seq[i-1:i+1] == 'AG': 
            a += 1
            accs[a] = i
    return d, a, dons, accs

in the revised version, the gtag sites use a dictionary for extracting informaton   

> dons = { `index`(int) : `donor site`(int)...}   
> accs = { `index`(int) : `accep site`(int)...}   

it would generate better performance when extracting ideal output and faster through looping

In [ ]:
flank = 100
minex = 50
minin = 50
d, a, dons, accs = gtag_sites(seq, flank, minex)

for d1 in range(1, d+1):
    if dons[d1] < minex + flank + 1: continue
    
    for a1 in range(1, a+1):
        if accs[a1] < dons[d1] + minin: continue

        for d2 in range(d1+1, d+1):
            if dons[d2] < accs[a1] + minex: continue

            for a2 in range(a1+1, a+1):
                if accs[a2] < dons[d2] + minin: continue

                for d3 in range(d2+1, d+1):
                    if dons[d3] < accs[a2] + minex: continue
                    
                    for a3 in range(a2+1, a+1):
                        if accs[a3] < minin + dons[d3]: continue
                        if accs[a3] + flank + minex > len(seq):  continue
                        print( dons[d1], accs[a1], dons[d2], accs[a2], dons[d3], accs[a3] )

Performance: 112.95s user 0.50s system 99% cpu 1:53.47 total

here is an simple test to proof baisc logic of the nest loop , they would generate all possible combination based on these idea

In [ ]:
for i in range(1,4+1):
    for j in range(i+1,5):
        print(i, j)

reflect on this version:
+ Pros   
  - Easier removing untargetted output   
    - Since everything is more specified, it is easier for removing small introns and exons   
    - we don't external def function there
  - Less time complexity
    - approximately `O(nlogn)` would be larger if max_exon comes more than 3   
    - since the combination n is collpasing through out the loop   
+ Cons
  - nobody wants such big truck of garbage code
    - max_exon could not even fix there
    - this could actually be fixed by using recursion method

## Idea 3
space complexity for time complexity

all previous idea make action on improving the overall `time complexity` for optimization   
the next trail for this idea is consuming more `space complexity` for better faster output    
below is the single proof for our idea

In [8]:
for i, j in zip( range(1,6), range(5,0,-1)):
    print(i, j)

1 5
2 4
3 3
4 2
5 1


In [ ]:
flank = 100
minex = 50
minin = 50
d, a, dons, accs = gtag_sites(seq, flank, minex)

for d1, rd1 in zip( range(1, d+1), range(d+1, 1, -1) ):
    if dons[d1] < minex + flank + 1: continue
    
    for a1 in range(1, a+1):
        if accs[a1] < dons[d1] + minin: continue

        for d2 in range(d1+1, d+1):
            if dons[d2] < accs[a1] + minex: continue

            for a2 in range(a1+1, a+1):
                if accs[a2] < dons[d2] + minin: continue

                for d3 in range(d2+1, d+1):
                    if dons[d3] < accs[a2] + minex: continue
                    
                    for a3 in range(a2+1, a+1):
                        if accs[a3] < minin + dons[d3]: continue
                        if accs[a3] + flank + minex > len(seq):  continue
                        print( dons[d1], accs[a1], dons[d2], accs[a2], dons[d3], accs[a3] )

seq = '''AGGTTTTCAGTCTGCGTATCTAAATCAGCATTAATAACAATGAGGCGAGGATAACAGCAA
TCGGGCCTCTAGTCCTTTTTGCTGGCCAAAGTGAACGTCAGCTAGGTAACGTTGCGCATT
CAAAGACGAGTAGTGTCGCGAAACAAATAGTTATCAGTGCTACAATTACATGCGCAACAA
TTGAGGAGTTAAGGCACTACCACGGAACAGAGACCGTGTGCCAGGAGAAGTGGTACGATT
GCTCCTGACAGTTGCCGACTCCGGGCGCATTGACTGTCACTTCGAAGTCATGGCACCGAG
GGGCTAGCCGGACCTCGGTAGCAGGCCCGTAGTCCTCCAGAGAGACACAGGTTTCGTACT
CCTAGTGAGAAATAATATCACCTGATCCCATGTTCACTAGGACCGTCACTAGCTTGCCTA
TGTGGCACAACGCAGGGAGCCAGATACAAATGAATCGCTAGCGGTTGCTGGCCCATGCCT
GGGCTATCTTAGTGACATTGGGCTGGAAAGGACAGTGCTGAGTATGGATGCCAGAAAAAA
GCCTAGACCGTGTGGGTGCTCGGACCGATCACCTCATTCACACACGACTTCATGCCGACT
TCCTCACGCTTGGTGGTATTCTCAGCGATATGGGTAGAATTCCCTGCTCAAGCCGGTGGT
TTATTGTGTTTGCGCTTCGGCAGGACGCTGCCTCGTGGGATCTCGATTTTAGCTGTGCCA
ATCAGCAATTTAGGAGAGCATATACATTGTTAACAACCCTATTTTACCTTACTCAGTAGG
TGCTACACTCTTTTACGTTAATGCCGTTATCCTCATAGACAGGTTACGTGGCGTGCGGCT
AGCGAGAGTGAGAAGTTGTCTTCACAGGATGTTTAGCTAAAGACTCACCGCAGCTCGTTC
TCAACGACGCTTGTTCACCGAAAAAAGTAAAGTGAGTTCCTACCCTTTGATAATATGTAT
TGGCGTGCTTGCGACGCCTGGCCTCAACAGATGCCCAGCTACCCTCATGGGGGGACTCGA
CACTCTACCCCTATTGGGACAGGTGAGACCTTTGGTCCTAGATGTTCTTCTCTCTCTAAG
CTCAGCTTATAATGCCGGTTAATTTGGCAACCCCCGTAAGTTGCCAGTTACTGATGTACG
GAAGGAGCCCTCCGTTGGGCCCGAAAACTCGATCGGTATGTCGGCCGATCCCCCCCCCAA
ACGAAAACAAGTTTCATTTAGTAGAGACGTACACTAACTGAAGTTGGGTCGACACGGCAT
TGCCGACAGTATTCTTTGTGTTCATCCTTCACGCAATAGCTCTAGGTCACCTATTGGGAG
TACGCAGTGACGATAAGTAGTGCACTAATTAGTACAGTCAGAGTCGTCATCGGAGCACTG
CAGCCTTGACGCGCTCATGCTTCTCTAGTCCTTGGACTTGGAGCCCACGTGTAGTGCTAG
GCACCCAAACAGTGATGTAGCCGAAACAATATATTTCCTGGATGACTCGTCCTTGAATCC'''